In [ ]:
# 加载必备库文件
import numpy as np

import mxnet as mx
from mxnet import nd
from mxnet import init
from mxnet import gluon
from mxnet import autograd

from mxnet.gluon import nn

In [ ]:
nn.Conv2D??

In [ ]:
res18_blk_num  = [2, 2, 2,  2] # 不是 bottleneck
res50_blk_num  = [3, 4, 6,  3]
res101_blk_num = [3, 4, 23, 3]
res152_blk_num = [3, 8, 36, 3]

# 为了便于参数导入，变量名最好和 gluoncv 的变量名相同
class BottleneckV1b(gluon.HybridBlock):
    """res 基本模块"""
    def __init__(self, channels, strides, isdownsample=False, **kwargs):
        super(BottleneckV1b, self).__init__(**kwargs)
        # 一个 bottleneck 内， 1*1 卷积 channel 扩大的倍数
        self.expansion = 4
        self.isdownsample = isdownsample
        self.strides = strides

        
        # bottletneck 内总是 1*1 conv -- 3*3 conv -- 1*1 conv
        self.conv1 = nn.Conv2D(channels=channels, kernel_size=(1,1), strides=(1,1),
                               padding=(0,0), groups=1, use_bias=False)
        self.bn1   = nn.BatchNorm() # use_global_stats=True 默认为 False
        self.relu  = nn.Activation('relu')
        
        self.conv2 = nn.Conv2D(channels=channels, kernel_size=3, strides=self.strides,
                               padding=(1,1), groups=1, use_bias=False)
        self.bn2   = nn.BatchNorm()
        
        self.conv3 = nn.Conv2D(channels=channels * self.expansion, kernel_size=(1,1), strides=(1,1),
                               padding=(0,0), groups=1, use_bias=False)
        self.bn3   = nn.BatchNorm()
        
        if self.isdownsample:
            self.downsample = nn.HybridSequential()
            self.downsample.add(nn.Conv2D(channels=channels * self.expansion, kernel_size=(1,1), strides=self.strides,
                                         padding=(0,0), groups=1, use_bias=False),
                                nn.BatchNorm())


    def hybrid_forward(self, F, x):
        residual = self.relu(self.bn1(self.conv1(x)))
        residual = self.relu(self.bn2(self.conv2(residual)))
        residual = self.bn3(self.conv3(residual))

        if self.isdownsample:
            x = self.downsample(x)

        x = x + residual
        out = self.relu(x)

        return out


In [ ]:
net = BottleneckV1b(64, (2,2), isdownsample=True)
x = nd.random.uniform(shape=(1,1,224,224))
net.initialize(init.Xavier())
y = net(x)

In [ ]:
class ResNet(gluon.HybridBlock):
    """ Pre-trained ResNetV1b Model, which produces the strides of 8
    featuremaps at conv5.

    Parameters
    ----------
    block : Block
        Class for the residual block. Options are BasicBlockV1, BottleneckV1.
    layers : list of int
        Numbers of layers in each block
    classes : int, default 1000
        Number of classification classes.
    norm_layer : object
        Normalization layer used (default: :class:`mxnet.gluon.nn.BatchNorm`)
        Can be :class:`mxnet.gluon.nn.BatchNorm` or :class:`mxnet.gluon.contrib.nn.SyncBatchNorm`.
    last_gamma : bool, default False
        Whether to initialize the gamma of the last BatchNorm layer in each bottleneck to zero.
    deep_stem : bool, default False
        Whether to replace the 7x7 conv1 with 3 3x3 convolution layers.
    avg_down : bool, default False
        Whether to use average pooling for projection skip connection between stages/downsample.
    final_drop : float, default 0.0
        Dropout ratio before the final classification layer.
    use_global_stats : bool, default False
        Whether forcing BatchNorm to use global statistics instead of minibatch statistics;
        optionally set to True if finetuning using ImageNet classification pretrained models.
    """
    # pylint: disable=unused-variable
    def __init__(self, block_nums, channels, strides, classes=1000, **kwargs):
        super(ResNet, self).__init__(**kwargs)

        self.features = nn.HybridSequential()
        self.features.add(nn.Conv2D(channels=64, kernel_size=(7,7), strides=(2,2),
                                    padding=(3,3), groups=1, use_bias=False))
        self.features.add(nn.BatchNorm())
        self.features.add(nn.Activation('relu'))
        self.features.add(nn.MaxPool2D(pool_size=(3,3), strides=(2,2), padding=(1,1)))
        
        # block_nums = [3, 4, 6, 3]
        # channels = [64, 128, 256, 512]
        # strides = [(1,1), (2,2), (2,2), (2,2)]
        for i in range(len(block_nums)):
            blk = nn.HybridSequential()
            for num in range(block_nums[i]) :
                if num == 0:
                    bottleneck = BottleneckV1b(channels[i], strides[i], isdownsample=True)
                else:
                    bottleneck = BottleneckV1b(channels[i], (1,1), isdownsample=False)
                blk.add(bottleneck)
            self.features.add(blk)

        self.avgpool = nn.GlobalAvgPool2D()
        self.out = nn.Dense(classes)


    def hybrid_forward(self, F, x):
        feature = self.features(x)
        out = self.avgpool(feature)
        out = self.out(out)
        return out

block_nums = [3, 4, 6, 3]
channels = [64, 128, 256, 512]
strides = [(1,1), (2,2), (2,2), (2,2)]

In [ ]:
net = ResNet(block_nums, channels, strides)
x = nd.random.uniform(shape=(1,1,224,224))
net.initialize(init.Xavier())
y = net(x)

In [ ]:
print(net)